In [ ]:
import os
import random

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import lightgbm as lgb
import tensorflow as tf
from tensorflow.keras import Sequential,Model
from tensorflow.keras.layers import Dense,Conv2D,Flatten,Dropout, Input, Concatenate, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from PIL import Image
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE  
img_size = 224
channels = 3
Batch_size = 64

# Directory for dataset

train_dir = "/kaggle/input/petfinder-pawpularity-score/train/"
test_dir = "/kaggle/input/petfinder-pawpularity-score/test/"

def seed_everything():
    np.random.seed(123)
    random.seed(123)
    tf.random.set_seed(123)
    os.environ["TF_CPP_MIN_LOG_LEVEL"] = '2'
    os.environ['PYTHONHASHSEED'] = str(123)

seed_everything()

In [ ]:
# Reading dataset train, test in df and df_test respectively

df = pd.read_csv("/kaggle/input/petfinder-pawpularity-score/train.csv")
df_test = pd.read_csv("/kaggle/input/petfinder-pawpularity-score/test.csv")
Id = df_test["Id"].copy()


# Converting Id column for taking images

df["Id"] = df["Id"].apply(lambda x : "/kaggle/input/petfinder-pawpularity-score/train/" + x + ".jpg")
df_test["Id"] = df_test["Id"].apply(lambda x : "/kaggle/input/petfinder-pawpularity-score/test/" + x + ".jpg")

In [ ]:
# Defining functions for reading and augmentation of images
# A seperate function for creating dataset

# Augmenting the image
def image_preprocess(is_labelled):  
    def augment(image):
        image = tf.image.random_flip_left_right(image)
#         image = tf.image.random_flip_up_down(image)
        image = tf.image.random_saturation(image, 0.95, 1.05)
        image = tf.image.random_contrast(image, 0.95, 1.05)
        return image
    
    def can_be_augmented(img, label):
        return augment(img), label
    
#   If record has label both image and lable will be returned
    return can_be_augmented if is_labelled else augment



# Reading and rescaling images
def image_read(is_labelled):
    def decode(path):
        image = tf.io.read_file(path)
        image = tf.image.decode_jpeg(image, channels=channels)
        image = tf.cast(image, tf.float32)
        image = tf.image.resize(image, (img_size, img_size))
        image = tf.keras.applications.efficientnet.preprocess_input(image) 
        return image
    
    def can_be_decoded(path, label):
        return decode(path), label
    
#   If record has label both image and lable will be returned

    return can_be_decoded if is_labelled else decode


# Creating the dataset
def create_dataset(df, batch_size, is_labelled = False, augment = False, shuffle = False):
    image_read_fn = image_read(is_labelled)
    image_preprocess_fn = image_preprocess(is_labelled)
    
    if is_labelled:
        dataset = tf.data.Dataset.from_tensor_slices((df["Id"].values, df["Pawpularity"].values))
    else:
        dataset = tf.data.Dataset.from_tensor_slices((df["Id"].values))
    
    dataset = dataset.map(image_read_fn, num_parallel_calls=AUTOTUNE)
    dataset = dataset.map(image_preprocess_fn, num_parallel_calls=AUTOTUNE) if augment else dataset
    dataset = dataset.shuffle(1024, reshuffle_each_iteration=True) if shuffle else dataset
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

In [ ]:
trn = df.iloc[:9000]
val = df.iloc[9001:]

train = create_dataset(trn, Batch_size, is_labelled = True, augment = True, shuffle = True)
validation = create_dataset(val, Batch_size, is_labelled = True, augment = False, shuffle = False)
test = create_dataset(df_test, Batch_size, is_labelled = False, augment = False, shuffle=False)

In [ ]:
img_mod = "/kaggle/input/keras-applications-models/EfficientNetB0.h5"
efnet = tf.keras.models.load_model(img_mod)

# Layers of efficientnet will not be trained
efnet.trainable = False

In [ ]:
model = Sequential([
    Input(shape=(img_size, img_size, channels)),
    efnet,
    BatchNormalization(),
    Dropout(0.2),
    Dense(units = 64, activation="relu"),
    Dense(units = 1, activation="relu")
])

In [ ]:
# Early stopping helps as it stops training if val_loss(validation score) does not decrease.

early_stopping = EarlyStopping(patience = 5,restore_best_weights=True)


lr_schedule = ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=100, decay_rate=0.96,
    staircase=True)

In [ ]:
model.compile(loss="mse", 
              optimizer = tf.keras.optimizers.Adam(learning_rate = lr_schedule), 
              metrics=[tf.keras.metrics.RootMeanSquaredError()])

predictor = model.fit(train,
                      epochs=20, 
                      validation_data = validation,
                      callbacks=[early_stopping])

In [ ]:

pred = model.predict(test)

final=pd.DataFrame()
final['Id']=Id
final['Pawpularity']=pred
final.to_csv('submission.csv',index=False)